In [ ]:
from itertools import combinations
from pysat.solvers import Glucose3
import random
import time

class ToCnf:
    def __init__(self, board):
        self.board = board
        self.KB = []
        self.subBoard = [[None for _ in range(
            len(board[0]))] for _ in range(len(board))]
        for i in range(len(board)):
            for j in range(len(board[0])):
                if board[i][j] != 0:
                    self.subBoard[i][j] = False
                    position = self.indexToPosition(i, j)
                    self.KB.append([-position])

    def indexToPosition(self, i, j):
        return i * len(self.board[1]) + j+1

    def positionToIndex(self, index):
        index -= 1
        i = index // len(self.board[1])
        j = index % len(self.board[1])
        return i, j

    def getAdjacent(self, i, j):
        m = len(self.board)
        n = len(self.board[0])
        adjacent_positions = []

        directions = [(-1, -1), (-1, 0), (-1, 1), (0, -1),
                      (0, 1), (1, -1), (1, 0), (1, 1), (0, 0)]

        for di, dj in directions:
            if 0 <= i + di < m and 0 <= j + dj < n:
                adjacent_positions.append((i + di, j + dj))

        return adjacent_positions

    def getUnassignAdjacent(self, i, j):
        m = len(self.board)
        n = len(self.board[0])
        adjacent_positions = []

        directions = [(-1, -1), (-1, 0), (-1, 1), (0, -1),
                      (0, 1), (1, -1), (1, 0), (1, 1), (0, 0)]

        for di, dj in directions:
            if 0 <= i + di < m and 0 <= j + dj < n and self.subBoard[i+di][j+dj] == None:
                adjacent_positions.append((i + di, j + dj))

        return adjacent_positions

    def caclUnAssign(self, adjacent):
        count = 0
        for i in range(len(adjacent)):
            if self.subBoard[adjacent[i][0]][adjacent[i][1]] == None:
                count += 1
        return count

    def calcMineRemain(self, i, j, adjacent):
        total = self.board[i][j]
        for i in range(len(adjacent)):
            if self.subBoard[adjacent[i][0]][adjacent[i][1]] == True:
                total -= 1
        return total



    def generateCNF(self, i, j):
        adjacent = self.getAdjacent(i, j)
        unAssign = self.getUnassignAdjacent(i, j)
        n = self.caclUnAssign(adjacent)
        k = self.calcMineRemain(i, j, adjacent)
        unAsgn = []
        for i in unAssign:
            unAsgn.append(self.indexToPosition(i[0], i[1]))
        if (k == 0):
            for i in unAsgn:
                self.KB.append([-i])
                index = self.positionToIndex(i)
                self.subBoard[index[0]][index[1]] = False
            return
        elif (k+1 > n):
            for i in unAsgn:
                self.KB.append([i])
                index = self.positionToIndex(i)
                self.subBoard[index[0]][index[1]] = True
            return

        tmp = list(combinations(unAsgn, k+1))
        U = list()
        for i in tmp:
            l = []
            for j in i:
                l.append(-j)
            self.KB.append(l)

        tmp = list(combinations(unAsgn, n-k+1))
        for i in tmp:
            self.KB.append(i)

    def pySAT(self):
        g = Glucose3()
        for clause in self.KB:
            g.add_clause(clause)

        if not g.solve():
            print('No solution')
        else:
            model = g.get_model()
            print(model)

    def generateKB(self):
        for i in range(len(self.board)):
            for j in range(len(self.board[0])):
                if self.board[i][j] != 0:
                    self.generateCNF(i, j)
        return self.KB

class SATsolver:
    def __init__(self,KB,board,maxGen,populationSize,mutatedRate):
        self.board=board
        self.assigned=[]
        # self.KB=KB
        # self.variables=[]
        self.KB,self.variables=self.simplyfiedKB(KB,board)
        for clause in self.KB:
            if len(clause)==1:
                self.KB,self.variables=self.simplyfiedKB(KB,board)
        self.maxGen=maxGen
        self.size=populationSize
        self.population=self.generatePopulation(populationSize)
        self.mutateRate=mutatedRate


    def generatePopulation(self,populationSize):
        population=[]
        for i in range (populationSize):
            chromosome={}
            assignment=[]
            for var in self.variables:
                random_boolean = random.choice([-1, 1])
                assignment.append(var*random_boolean)
            chromosome['assignment']=assignment
            chromosome['fitness']=self.fitnessFunction(assignment)
            population.append(chromosome)
        return population

    def indexToPosition(self, i, j):
        return i * len(self.board[1]) + j+1

    def positionToIndex(self, index):
        index -= 1
        i = index // len(self.board[1])
        j = index % len(self.board[1])
        return i, j

    def simplyfiedKB(self,KB,board):
        newKB=[]
        tmp=[]
        variables=[]
        for clause in KB:
            if len(clause)==1:
                tmp.append(clause[0])
                self.assigned.append(clause[0])
        for i in range(len(board)):
            for j in range(len(board[0])):
                if board[i][j] != 0:
                    position = self.indexToPosition(i, j)*-1
                    tmp.append(position)
                    self.assigned.append(position)


        for clause in KB:
            added=True
            for var in clause :
                # Satisfied clause
                if var in tmp :
                    added=False
                    break
                elif -var in tmp:
                    idx = clause.index(var)
                    clause = clause[:idx] + clause[idx+1:]
            if(added and len(clause)>0):
                clause=list(clause)
                newKB.append(clause)
                for var in clause:
                    if abs(var) not in variables:
                        variables.append(abs(var))

        return newKB,variables

    # Take an assignment
    def fitnessFunction(self,gen):
        satisfied=0
        for clause in self.KB:
            for var in clause:
                if var in gen:
                    satisfied+=1
                    break

        return satisfied

    def calcProbability(self):
        probs=[]
        total=0
        for chronosome in self.population:
            total+=chronosome['fitness']
        for chronosome in self.population:
            prob=chronosome['fitness']/total
            probs.append(prob)

        return probs

    def selectNumbers(self, nums, probabilities, n):
        selected_nums = random.choices(nums, probabilities, k=n)
        return selected_nums

    def isGoal(self):
        for chro in self.population:
            if chro['fitness']==len(self.KB):
                return (chro['assignment']+self.assigned)
        return None

    def crossover(self,dad,mom):
        assignDad=dad['assignment']
        assignMom=mom['assignment']
        crosspoint = random.randint(1, len(assignDad)-2)
        newDad=assignDad[:crosspoint+1]+assignMom[crosspoint+1:]
        newMom=assignMom[:crosspoint+1]+assignDad[crosspoint+1:]
        dad['assignment']=newDad
        mom['assignment']=newMom
        dad['fitness']=self.fitnessFunction(newDad)
        mom['fitness']=self.fitnessFunction(newMom)

        return dad,mom

    def mutation(self,chro):
        if random.random()<=self.mutateRate:
            assign=chro['assignment']
            index=random.choice(range(len(assign)-1))
            assign[index]*=-1
            chro['assignment']=assign
        return chro

    def GA(self):
        for _ in range(self.maxGen):
            probabilities=self.calcProbability()
            selected=self.selectNumbers(self.population,probabilities,self.size)
            new_population=[]

            for i in range(0,self.size,2):
                dad=selected[i]
                mom=selected[i+1]
                new_dad,new_mom=self.crossover(dad,mom)
                new_dad=self.mutation(new_dad)
                new_mom=self.mutation(new_mom)
                new_population.append(new_dad)
                new_population.append(new_mom)
            self.population=new_population

            solution=self.isGoal()
            if solution is not None:
                return solution

        return None

    def convert_to_board(self,assignment):
        board=self.board
        for i in range(len(self.board)):
            for j in range(len(self.board[0])):
                position=self.indexToPosition(i,j)
                if position in assignment:
                    board[i][j]='X'
                elif board[i][j]==0:
                    board[i][j]='-'

        return board


def read_2d_array_from_file(file_path):
    try:
        with open(file_path, 'r') as file:
            lines = file.readlines()

        array_2d = [list(map(int, line.strip().split())) for line in lines]

        return array_2d
    except FileNotFoundError:
        print("File not found")
        return None
    except Exception as e:
        print("Error:", e)
        return None

def save_2d_array_to_file(array_2d, file_path):
    try:
        with open(file_path, 'w') as file:
            for row in array_2d:
                row_str = ' '.join(map(str, row))
                file.write(row_str + '\n')
        print("Save success.")
    except Exception as e:
        print("Error:", e)
# board=[
#     [2,0,0],
#     [0,0,0],
#     [0,0,0]
# ]
# board=[
#     [0,0,2,0],
#     [2,0,2,0],
#     [0,3,0,0],
#     [0,0,0,1]
# ]
# board=[
#     [1,0,2,0,0,0],
#     [0,0,0,0,0,0],
#     [1,0,3,0,0,0],
#     [0,0,0,2,2,0],
#     [0,2,0,0,0,0],
#     [0,2,1,0,0,0]
# ]


board=read_2d_array_from_file('input.txt')
mine_sweeper = ToCnf(board)
output = mine_sweeper.generateKB()
# mine_sweeper.pySAT()
solver=SATsolver(output,board,100000,10,0.2)
start_time = time.time()
assignment=solver.GA()
end_time = time.time()
execution_time = end_time - start_time
if assignment is None:
    print("No solution")
else:
    print(assignment)
    solution=solver.convert_to_board(assignment)
    save_2d_array_to_file(solution,'output.txt')
    print("Time executing: {:.6f} second".format(execution_time))



ModuleNotFoundError: ignored